## **RA1: table 221118**

Dong Gyun Ko <br>

last updated: november 18, 2022 <br>

In [1]:
# import libraries

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import datetime as dt
import os
import warnings
import requests # python 3.6

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# set the working directory

os.chdir('/content/gdrive/MyDrive/Colab Notebooks/RA1_data')

## **1. import data_221115**

In [3]:
# import the data

data_221115 = pd.read_excel('data_221115.xlsx')
data_221115['quarter'] = pd.to_datetime(data_221115['quarter'])
data_221115['quarter'] = data_221115['quarter'].dt.to_period('Q')

In [4]:
# quarterly dataset 221020

data_221115 = data_221115.sort_values(by=['isocode', 'quarter']).drop(columns=['Unnamed: 0']).reset_index(drop=True)

data_221115

,country,isocode,quarter,region,income,oecd,g7,tradeopen,finopen_ci,finopen_ci_n,...,fxflex_fine,fxflex_coarse,ggdebt,ggcon,gdp,reer,ca,pcon,cds,inv
0,Argentina,ARG,1950Q1,7,2,NaN,NaN,NaN,NaN,NaN,...,14.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Argentina,ARG,1950Q2,7,2,NaN,NaN,NaN,NaN,NaN,...,14.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Argentina,ARG,1950Q3,7,2,NaN,NaN,NaN,NaN,NaN,...,14.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Argentina,ARG,1950Q4,7,2,NaN,NaN,NaN,NaN,NaN,...,14.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Argentina,ARG,1951Q1,7,2,NaN,NaN,NaN,NaN,NaN,...,14.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14495,South Africa,ZAF,2021Q2,5,2,NaN,NaN,55.219369,NaN,NaN,...,NaN,NaN,70.881640,2.236984e+11,1.165743e+12,80.196367,5.043574,7.696255e+11,187.5187,1.624898e+11
14496,South Africa,ZAF,2021Q3,5,2,NaN,NaN,57.412444,NaN,NaN,...,NaN,NaN,70.725171,2.239592e+11,1.124135e+12,78.671305,2.392012,7.426943e+11,215.0529,1.596462e+11
14497,South Africa,ZAF,2021Q4,5,2,NaN,NaN,59.447237,NaN,NaN,...,NaN,NaN,71.314406,2.240916e+11,1.125567e+12,75.720268,3.023261,7.469260e+11,205.5150,1.571342e+11
14498,South Africa,ZAF,2022Q1,5,2,NaN,NaN,61.192979,NaN,NaN,...,NaN,NaN,NaN,2.258330e+11,1.137366e+12,76.993295,2.612085,7.557754e+11,211.3849,1.615184e+11


## **2. python code**

In [5]:
data_221115.columns

Index(['country', 'isocode', 'quarter', 'region', 'income', 'oecd', 'g7',
       'tradeopen', 'finopen_ci', 'finopen_ci_n', 'finopen_fa', 'fxflex_fine',
       'fxflex_coarse', 'ggdebt', 'ggcon', 'gdp', 'reer', 'ca', 'pcon', 'cds',
       'inv'],
      dtype='object')

In [6]:
# function generating the sample period by country

# import data_221115 as data
data = data_221115

# stat fun
def period(var):

    for i in range(0, data['isocode'].unique().shape[0]):
  
        globals()['cond_isocode_{}'.format(i+1)] = (data['isocode'] == data['isocode'].unique()[i])
        globals()['df_isocode_{}'.format(i+1)] = data.loc[globals()['cond_isocode_{}'.format(i+1)]]
        globals()['df_isocode_count_T_{}'.format(i+1)] = globals()['df_isocode_{}'.format(i+1)].groupby(globals()['df_isocode_{}'.format(i+1)]['quarter']).count()

        globals()['cond_{}'.format(var) + '_{}'.format(i+1)] = (globals()['df_isocode_count_T_{}'.format(i+1)][var] == 0)
        globals()['df_isocode_count_T_{}'.format(i+1)] = globals()['df_isocode_count_T_{}'.format(i+1)].loc[~globals()['cond_{}'.format(var) + '_{}'.format(i+1)]]
        globals()['df_isocode_count_T_{}'.format(i+1)] = globals()['df_isocode_count_T_{}'.format(i+1)].reset_index()

        # missing
        if globals()['df_isocode_count_T_{}'.format(i+1)]['quarter'].unique().shape[0] == 0:

           globals()['tab_period_{}'.format(var) + '_{}'.format(i+1)] = pd.DataFrame(np.array([data['country'].unique()[i], 
                                                                                              data['isocode'].unique()[i],  
                                                                                              np.nan, 
                                                                                              np.nan]).reshape(1, -1)
                                                                                              ).rename(columns={0: 'country', 
                                                                                                                1: 'isocode', 
                                                                                                                2: 'period', 
                                                                                                                3: 'obs'}).reset_index(drop=True)

        # observed
        else:

           globals()['tab_period_{}'.format(var) + '_{}'.format(i+1)] = pd.DataFrame(np.array([data['country'].unique()[i], 
                                                                                              data['isocode'].unique()[i], 
                                                                                              str(globals()['df_isocode_count_T_{}'.format(i+1)]['quarter'].unique().min()) + '-' + str(globals()['df_isocode_count_T_{}'.format(i+1)]['quarter'].unique().max()),
                                                                                              globals()['df_isocode_count_T_{}'.format(i+1)][var].count()]).reshape(1, -1)
                                                                                              ).rename(columns={0: 'country', 
                                                                                                                1: 'isocode', 
                                                                                                                2: 'period', 
                                                                                                                3: 'obs'}).reset_index(drop=True)

    globals()['tab_period_{}'.format(var)] = pd.concat([globals()['tab_period_{}'.format(var) + '_{}'.format(1)], globals()['tab_period_{}'.format(var) + '_{}'.format(2)]])

    for j in range(3, data['isocode'].unique().shape[0]+1):
        
        globals()['tab_period_{}'.format(var)] = pd.concat([globals()['tab_period_{}'.format(var)], globals()['tab_period_{}'.format(var) + '_{}'.format(j)]])

    globals()['tab_period_{}'.format(var)] = pd.DataFrame(globals()['tab_period_{}'.format(var)]).reset_index(drop=True)

    return globals()['tab_period_{}'.format(var)]

In [7]:
# advanced

def period_var_advanced(var):

    advanced_list = ['AUS', 'AUT', 'BEL', 'CAN', 'CHE', 'CYP', 'CZE', 'DEU', 'DNK', 'ESP', 
                     'EST', 'FIN', 'FRA', 'GBR', 'GRC', 'HKG', 'IRL', 'ISR', 'ITA', 'JPN', 
                     'KOR', 'LTU', 'LUX', 'LVA', 'MLT', 'NLD', 'NOR', 'NZL', 'PRT', 'SGP', 
                     'SVK', 'SVN', 'SWE', 'USA']

    tab_period_var = period(var)

    for i in range(0, len(advanced_list)):
  
        globals()['cond_N{}'.format(i+1)] = (tab_period_var['isocode'] == advanced_list[i])

    tab_period_var_advanced = tab_period_var.loc[cond_N1 | cond_N2 | cond_N3 | cond_N4 | cond_N5 | cond_N6 | cond_N7 | cond_N8 | cond_N9 | cond_N10 |
                                                 cond_N11 | cond_N12 | cond_N13 | cond_N14 | cond_N15 | cond_N16 | cond_N17 | cond_N18 | cond_N19 | cond_N20 |
                                                 cond_N21 | cond_N22 | cond_N23 | cond_N24 | cond_N25 | cond_N26 | cond_N27 | cond_N28 | cond_N29 | cond_N30 |
                                                 cond_N31 | cond_N32 | cond_N33 | cond_N34]
 
    tab_period_var_advanced = tab_period_var_advanced.sort_values(by=['isocode']).reset_index(drop=True)

    return tab_period_var_advanced

In [8]:
# developing

def period_var_developing(var):

    developing_list = ['ARG', 'BGR', 'BRA', 'CHL', 'COL', 'HRV', 'HUN', 'IDN', 'IND', 'MEX', 
                       'POL', 'ROU', 'RUS', 'THA', 'TUR', 'ZAF']

    tab_period_var = period(var)

    for i in range(0, len(developing_list)):
  
        globals()['cond_N{}'.format(i+1)] = (tab_period_var['isocode'] == developing_list[i])

    tab_period_var_developing = tab_period_var.loc[cond_N1 | cond_N2 | cond_N3 | cond_N4 | cond_N5 | cond_N6 | cond_N7 | cond_N8 | cond_N9 | cond_N10 |
                                                   cond_N11 | cond_N12 | cond_N13 | cond_N14 | cond_N15 | cond_N16]

    tab_period_var_developing = tab_period_var_developing.sort_values(by=['isocode']).reset_index(drop=True)

    return tab_period_var_developing

## **3. sample period table**

### **3.1. tab_period_ggcon_advanced_221118**

In [9]:
period_var_advanced('ggcon')

,country,isocode,period,obs
0,Australia,AUS,1959Q3-2022Q1,251
1,Austria,AUT,1995Q1-2022Q2,110
2,Belgium,BEL,1995Q1-2022Q1,109
3,Canada,CAN,1961Q1-2022Q1,245
4,Switzerland,CHE,1980Q1-2022Q1,169
5,Cyprus,CYP,1995Q1-2022Q1,109
6,Czech Republic,CZE,1996Q1-2022Q1,105
7,Germany,DEU,1991Q1-2022Q1,125
8,Denmark,DNK,1995Q1-2022Q1,109
9,Spain,ESP,1995Q1-2022Q2,110


In [10]:
# total

period_var_advanced('ggcon')['obs'].astype(float).sum()

4788.0

### **3.2. tab_period_ggcon_developing_221118**

In [11]:
period_var_developing('ggcon')

,country,isocode,period,obs
0,Argentina,ARG,2004Q1-2022Q1,73
1,Bulgaria,BGR,1995Q1-2022Q1,109
2,Brazil,BRA,1996Q1-2022Q1,105
3,Chile,CHL,1996Q1-2022Q2,106
4,Colombia,COL,2005Q1-2022Q1,69
5,Croatia,HRV,1995Q1-2022Q1,109
6,Hungary,HUN,1995Q1-2022Q1,109
7,Indonesia,IDN,1990Q1-2022Q2,130
8,India,IND,1996Q2-2022Q1,104
9,Mexico,MEX,1993Q1-2022Q1,117


In [17]:
# total

period_var_developing('ggcon')['obs'].astype(float).sum()

1654.0

### **3.3. tab_period_gdp_advanced_221118**

In [18]:
period_var_advanced('gdp')

,country,isocode,period,obs
0,Australia,AUS,1959Q3-2022Q1,251
1,Austria,AUT,1995Q1-2022Q2,110
2,Belgium,BEL,1995Q1-2022Q1,109
3,Canada,CAN,1961Q1-2022Q2,246
4,Switzerland,CHE,1980Q1-2022Q1,169
5,Cyprus,CYP,1995Q1-2022Q1,109
6,Czech Republic,CZE,1996Q1-2022Q1,105
7,Germany,DEU,1991Q1-2022Q2,126
8,Denmark,DNK,1995Q1-2022Q1,109
9,Spain,ESP,1995Q1-2022Q2,110


In [19]:
# total

period_var_advanced('gdp')['obs'].astype(float).sum()

4813.0

### **3.4. tab_period_gdp_developing_221118**

In [20]:
period_var_developing('gdp')

,country,isocode,period,obs
0,Argentina,ARG,2004Q1-2022Q1,73
1,Bulgaria,BGR,1995Q1-2022Q1,109
2,Brazil,BRA,1996Q1-2022Q1,105
3,Chile,CHL,1996Q1-2022Q2,106
4,Colombia,COL,2005Q1-2022Q1,69
5,Croatia,HRV,1995Q1-2022Q1,109
6,Hungary,HUN,1995Q1-2022Q1,109
7,Indonesia,IDN,2000Q1-2022Q2,90
8,India,IND,1996Q2-2022Q1,104
9,Mexico,MEX,1993Q1-2022Q2,118


In [21]:
# total

period_var_developing('gdp')['obs'].astype(float).sum()

1576.0

### **3.5. tab_period_ca_advanced_221118**

In [22]:
period_var_advanced('ca')

,country,isocode,period,obs
0,Australia,AUS,1960Q1-2022Q2,250
1,Austria,AUT,1984Q1-2022Q2,154
2,Belgium,BEL,1989Q1-2022Q2,134
3,Canada,CAN,1961Q1-2022Q1,245
4,Switzerland,CHE,1985Q1-2022Q1,149
5,Cyprus,CYP,2001Q1-2007Q4,28
6,Czech Republic,CZE,1995Q1-2022Q1,109
7,Germany,DEU,1991Q1-2022Q1,125
8,Denmark,DNK,1989Q1-2022Q2,134
9,Spain,ESP,1989Q1-2022Q2,134


In [23]:
# total

period_var_advanced('ca')['obs'].astype(float).sum()

4378.0

### **3.6. tab_period_ca_developing_221118**

In [24]:
period_var_developing('ca')

,country,isocode,period,obs
0,Argentina,ARG,2004Q1-2022Q1,73
1,Bulgaria,BGR,1995Q1-2022Q1,109
2,Brazil,BRA,1996Q1-2022Q1,105
3,Chile,CHL,1996Q1-2021Q3,103
4,Colombia,COL,2005Q1-2022Q2,70
5,Croatia,HRV,1995Q1-2022Q1,109
6,Hungary,HUN,1995Q1-2022Q1,109
7,Indonesia,IDN,2004Q1-2022Q2,74
8,India,IND,1996Q2-2022Q1,104
9,Mexico,MEX,1993Q1-2022Q1,117


In [25]:
# total

period_var_developing('ca')['obs'].astype(float).sum()

1708.0

### **3.7. tab_period_reer_advanced_221118**

In [26]:
period_var_advanced('reer')

,country,isocode,period,obs
0,Australia,AUS,1980Q1-2022Q2,170
1,Austria,AUT,1979Q1-2022Q2,174
2,Belgium,BEL,1979Q1-2022Q2,174
3,Canada,CAN,1979Q1-2022Q2,174
4,Switzerland,CHE,1979Q1-2022Q2,174
5,Cyprus,CYP,1980Q1-2022Q2,170
6,Czech Republic,CZE,1990Q1-2022Q2,130
7,Germany,DEU,1979Q1-2022Q2,174
8,Denmark,DNK,1979Q1-2022Q2,174
9,Spain,ESP,1980Q1-2022Q2,170


In [27]:
# total

period_var_advanced('reer')['obs'].astype(float).sum()

5440.0

### **3.8. tab_period_reer_developing_221118**

In [28]:
period_var_developing('reer')

,country,isocode,period,obs
0,Argentina,ARG,1994Q1-2022Q2,114
1,Bulgaria,BGR,1993Q1-2022Q2,118
2,Brazil,BRA,1980Q1-2022Q2,170
3,Chile,CHL,1980Q1-2022Q2,170
4,Colombia,COL,1980Q1-2022Q2,170
5,Croatia,HRV,1994Q1-2022Q2,114
6,Hungary,HUN,1990Q4-2022Q2,127
7,Indonesia,IDN,1994Q1-2022Q2,114
8,India,IND,1994Q1-2022Q2,114
9,Mexico,MEX,1980Q1-2022Q2,170


In [29]:
# total

period_var_developing('reer')['obs'].astype(float).sum()

2151.0

## **4. export the table**

In [17]:
# set the working directory

os.chdir('/content/gdrive/MyDrive/Colab Notebooks/RA1_data/table_221118')

In [18]:
# export the table

var_list = ['ggcon', 'gdp', 'ca', 'reer']

# advanced
for k in range(0, len(var_list)):

    period_var_advanced(var_list[k]).to_excel(excel_writer='tab_period_{}'.format(var_list[k]) + '_advanced_221118.xlsx')

# developing
for k in range(0, len(var_list)):

    period_var_developing(var_list[k]).to_excel(excel_writer='tab_period_{}'.format(var_list[k]) + '_developing_221118.xlsx')